In [767]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
from JoyStick import Joystick
from Background import Background
#from Character import Character


In [768]:
class Character:
  def __init__(self,life):
    # 캐릭터에 사용할 이미지
    self.cat_front = Image.open('images/character/cat_front.png').convert('RGBA')
    self.cat_left = Image.open('images/character/cat_left.png').convert('RGBA')
    self.cat_right = Image.open('images/character/cat_right.png').convert('RGBA')
    self.cat_angry = Image.open('images/character/cat_angry.png').convert('RGBA')
    # 이미지 크기 줄이기 
    self.cat_front_resized = self.cat_front.resize((70, 70)) 
    self.cat_left_resized = self.cat_left.resize((70, 70)) 
    self.cat_right_resized = self.cat_right.resize((70, 70)) 
    self.cat_angry_resized = self.cat_angry.resize((70, 70)) 
    # 캐릭터 위치
    self.x_position = 120
    self.y_position = 180
    self.width = 70
    self.height = 70
    #self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 +20])
    self.speed = 5
    self.state= None

    # 캐릭터 목숨
    self.life = life
    self.life_list=[]
    # 목숨은 3개, 리스트에 이미지를 넣는다.
    self.life_image = Image.open('./images/character/heart.png').convert('RGBA')
    self.life_image_resized = self.life_image.resize((20,20))
    self.life_list.append(self.life_image_resized)
    self.life_list.append(self.life_image_resized)
    self.life_list.append(self.life_image_resized)

    self.score = 0

   # 위 아래로는 움직이지 않음 
  def move(self,up_down,left_right):
      if up_down=='up_pressed':
           pass
      elif up_down == 'down_pressed':
           pass
      elif left_right == 'left_pressed':
          self.x_position -= self.speed
      elif left_right == 'right_pressed':
          self.x_position += self.speed

In [769]:
class Fish:
  def __init__(self,x1,y1,x2,y2,stage,speed):
    self.fish_1 = Image.open('images/character/fish.png').convert('RGBA')
    self.fish2 = Image.open('images/character/fish2.png').convert('RGBA')
    self.fish_resized = self.fish_1.resize((50, 50)) 
    self.fish2_resized = self.fish2.resize((40, 40))
    #물고기가 내려오는 속도
    self.speed = speed 
    # 물고기의 위치
    self.x1=x1
    self.y1=y1
    self.x2=x1+50
    self.y2=y1+50
    self.col_count = 0
    self.is_collided = False
    self.stage=stage

    if(stage%10 == 1):
      self.speed = 5    
      print('stage : 1')
    elif (stage % 10 == 2):
      self.speed = 5   
      print('stage : 2')
    elif (stage % 10 == 3):
      self.speed = 5    
      print('stage : 3')
    elif (stage % 10 ==4 ):
      self.speed = 5
      print('stage : 4')
  
# 물고기 떨어지는 거
  def descend(self):
        #y만 self.speed 만큼 + 하면서 내려오게 하면 됨
      if self.y1 < 240 :
          self.y1 += self.speed
          self.y2 += self.speed
      else:
            self.y1 = 0
            self.y2 = 50  # Assuming fish height is 50
            self.x1 = random.randrange(0, 240)
            self.x2 = self.x1 + 50

  def collision_check(self, cat):
      if (
          cat.x_position + cat.width >= self.x1 and 
          cat.x_position <= self.x2 and 
          cat.y_position + cat.height  >= self.y1 and 
          cat.y_position +50 <= self.y2
          ):
          print("get score!")
          return 1
      else:
         return 0
                
  def speed_up(self):
        self.speed += 100
        
  def reset_collision(self):
        self.is_collided = False            

In [770]:
class Fishes:
  def __init__(self, stage):
    self.fish_list = []
    self.stage = stage

    if stage == 1:
            for i in range(3):
                x1 = random.randrange(0, 240)
                y1 = 0
                x2 = random.randrange(0, 240)
                y2 = 0
                self.fish_list.append(Fish(x1, y1,x2,y2,stage, 5))
  


In [771]:
class Bone:
  def __init__(self,x1,y1,x2,y2):
     self.image = Image.open('images/character/bone.png').convert("RGBA")
     self.image_resized = self.image.resize((50, 50))
    #물고기가 내려오는 속도
    # 물고기의 위치
     self.x1=x1
     self.y1=y1
     self.x2=x1+50
     self.y2=y1+50
     self.is_collided = False
     self.speed = 5

  def descend(self):
        #y만 self.speed 만큼 + 하면서 내려오게 하면 됨
      if self.y1 < 240 :
          self.y1 = self.y1 + self.speed
          self.y2 = self.y2 + self.speed
      else:
            self.y1 = 0
            self.y2 = 50  # Assuming fish height is 50
            self.x1 = random.randrange(0, 240)
            self.x2 = self.x1 + 50
        
  
# 사각형 충돌
  def collision_check(self, cat):
      if (
          cat.x_position + 20  >= self.x1 and 
          cat.x_position <= self.x2 and 
          cat.y_position + cat.height  >= self.y1 and 
          cat.y_position +50 <= self.y2
          ):
          return 1
      else:
         return 0


In [772]:
class Bones:
  def __init__(self,stage):
    self.bone_list = []
    self.stage = stage
  
    if stage == 1:
            for i in range(3):
                x1 = random.randrange(0, 240)
                y1 = 0
                x2 = random.randrange(0, 240)
                y2 = 0
                self.bone_list.append(Bone(x1, y1, x2, y2))

In [773]:
life = 3
hit=0
left_right = 'None'
k=0
score= 0
new_stage = 1
fish_index = [0, 1, 2, 3, 4, 5, 6, 7]

# joystick 객체
joystick = Joystick()
background = Background()

# 캐릭터 객체 생성
cat = Character(life)
delete_cat = [] #지워지는 적들 index 리스트에 저장

# # 물고기를 넣을 리스트
# fishes=[]
# speed =  random.randrange(0,10)
# fish = Fish(0, 0, 0, 0, speed)
# fishes.append(fish)
stage = 1

# 물고기 객체 생성
fishes = Fishes(stage)
bones = Bones(stage)


# Get drawing object to draw on image
image = Image.new("RGBA", (joystick.width, joystick.height))
draw = ImageDraw.Draw(image)
# clear display
draw.rectangle((0, 0, joystick.width, joystick.height), outline=0, fill=(255, 0, 0))
joystick.disp.image(image)


# 폰트 설정하기
fnt = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)

stage : 1
stage : 1
stage : 1


In [774]:
#background = Image.open('images/background/background1.png')
#background = background.resize((240,240))
# for i in range(10):
#   background = Background().backgroundList[i]
#   background = background.resize((240,240))
total_backgrounds=len(background.backgroundList)
current_background_index = 0



In [775]:
while True:
    up_down = 'None'
    left_right = 'None'
    if not joystick.button_U.value:
         up_down = 'up_pressed'
    if not joystick.button_D.value:
         up_down = 'down_pressed'
    if not joystick.button_L.value:
        left_right = 'left_pressed'
    if not joystick.button_R.value:
        left_right = 'right_pressed'
        
    time.sleep(0.01)
    current_background_index = (current_background_index + 1) % 10
    current_background = background.backgroundList[current_background_index]
    current_background = current_background.resize((240, 240))
    image.paste(current_background, (0, 0),current_background)

    # 목숨 몇개 남았는 지 보여주기, 만약 hit이 생겼다면 리스트에서 하나씩 빼면 됨 -> 나중에 구현해보자
    # 3개의 목숨이 다 보이게 할려면 이미지가 붙을 위치를 다르게 해야하기 때문에 j의 값을 바꿔야함
    j=0
    for i in range(cat.life):
        image.paste(cat.life_list[i],(j,0),cat.life_list[i])
        j+=15
    
    if(cat.life<=0):
        while True:
         rcolor = tuple(int (x * 255) for x in hsv_to_rgb(random.random(), 1, 1))
         draw.text((30, 70), "Game Over", font = fnt, fill = rcolor)
         draw.text((50, 100), "Game Over", font = fnt, fill = rcolor)
         draw.text((70, 130), "Game Over", font = fnt, fill = rcolor)    #game에서 클리어 하지 못할 시 형형색색으로 F학점을 맞았다고 해줌
         joystick.disp.image(image)
    
     
    # if new_stage == 0: #새로운 스테이지 시 지울 enemies 초기화 후 랜덤으로 뽑기
    #     delete_enemy = random.sample(fish_index, 2)
    #     new_stage = 1 #1일땐 old stage
    
    for i in range(len(fishes.fish_list)):
          fishes.fish_list[i].descend()
          if fishes.fish_list[i].y1 == 0:
              new_fish = Fish(random.randrange(0, 240), 0, random.randrange(0, 240), 0,fishes.fish_list[i].stage, fishes.fish_list[i].speed)
              fishes.fish_list[fishes.fish_list.index(fishes.fish_list[i])] = new_fish
          x_position = fishes.fish_list[i].x1
          y_position =  fishes.fish_list[i].y1
          image.paste (fishes.fish_list[i].fish_resized, (x_position, y_position), fishes.fish_list[i].fish_resized)

    for i in range(len(bones.bone_list)):
          bones.bone_list[i].descend()
          if bones.bone_list[i].y1 == 0:
              new_bone = Bone(random.randrange(0, 240), 0, random.randrange(0, 240), 0)
              bones.bone_list[i] = new_bone
          x_position = bones.bone_list[i].x1
          y_position =  bones.bone_list[i].y1
          image.paste (bones.bone_list[i].image_resized, (x_position, y_position), bones.bone_list[i].image_resized)
    
    # for i in range(len(bones.bone_list)):
    #  bones.bone_list[i].descend()
    #  x_position = bones.bone_list[i].x1
    #  y_position =  bones.bone_list[i].y1
    #  image.paste(bones.bone_list[i].image_resized,(x_position, y_position),bones.bone_list[i].image_resized )
         
        
    # 움직이게 만들기 
    cat.move(up_down, left_right)
    # 가만히 있을 때 이미지표시
    image.paste(cat.cat_front_resized, (cat.x_position,cat.y_position),cat.cat_front_resized)
    # 왼쪽 오른쪽으로 움직일 때 다른 이미지로 보이도록 함.
    if(left_right =='left_pressed') :
        image.paste(cat.cat_left_resized, (cat.x_position,cat.y_position),cat.cat_left_resized)
    elif(left_right == 'right_pressed'):
         image.paste(cat.cat_right_resized, (cat.x_position,cat.y_position),cat.cat_right_resized)
    
    # for fish in fishes:
    #     image.paste(fish,(0,0),fish)
    draw.text((0, 10), "score:", font = fnt, fill = (255,255,255))
    draw.text((60, 10), str(score), font = fnt, fill = (255,255,255))

    # 물고기와의 충돌 체크
    # 충돌 되면 score 점수가 올라감
    # score점수가 100이 되면 stage가 하나 올라감
    for i in range(len(fishes.fish_list)):
        if not fishes.fish_list[i].is_collided:  # 이미 충돌이 확인된 물고기는 무시
            get_score = fishes.fish_list[i].collision_check(cat)
            if get_score == 1:
                score +=10
                fishes.fish_list[i].is_collided = True

    for i in range(len(bones.bone_list)):
        if not bones.bone_list[i].is_collided:  # 이미 충돌이 확인된 물고기는 무시
            hit = bones.bone_list[i].collision_check(cat)
            if hit == 1:
                life -= 1
                bones.bone_list[i].is_collided = True
                cat.life_list.pop()
                cat.life -= 1
                image.paste(cat.cat_angry_resized, (cat.x_position,cat.y_position),cat.cat_angry_resized)
                print("collison!!")          

    if(str(score) == "40"):
         stage+=1
         print("stage:",stage)
         for fish in fishes.fish_list:
             fish.speed_up()  # 물고기 속도 증가
             score = 0  # score 초기화
             if(stage==2):
                 new_fish2 = fishes.fish_list[i]


    joystick.disp.image(image)


get score!
get score!
stage : 1
stage : 1
stage : 1
get score!
collison!!
collison!!
stage : 1
stage : 1
stage : 1
get score!
get score!
collison!!


KeyboardInterrupt: 